In [ ]:
import torch

torch.has_mps

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='./gpt2-hvac', tokenizer='gpt2')

In [ ]:
num_inputs = 11
num_outputs = 7

all_prediction_errors = []

# function to get mean of columns of list of lists
def mean_columns(list_of_lists):
    mean_list = []
    for i in range(len(list_of_lists[0])):
        column_mean = 0
        for j in range(len(list_of_lists)):
            column_mean += list_of_lists[j][i]
        mean_list.append(column_mean / len(list_of_lists))
    return mean_list

with open('test_dataset.txt', 'r') as f:
    for line in f:
        while True:
            try:
                parts = line.strip().split(',')
                numbers = [float(x) for x in parts]
                inputs = numbers[:num_inputs]
                outputs = numbers[num_inputs:]
                generated_text = generator(','.join([str(x) for x in inputs]) + ',', max_new_tokens=128, return_full_text=False)[0]['generated_text']
                predicted_outputs = [float(x.split()[0]) for x in generated_text.split(',')][:len(outputs)]
                prediction_errors = [abs(x - y) for x, y in zip(outputs, predicted_outputs)]
                all_prediction_errors.append(prediction_errors)
                mean_all_prediction_errors = mean_columns(all_prediction_errors)
                print('Current mean output errors:', mean_all_prediction_errors)
                print('Current total mean error:', sum(mean_all_prediction_errors) / len(mean_all_prediction_errors))
                break
            except (IndexError, ValueError) as error:
                print(error)
                print('Trying again')
            